<h1>Modelling</h1>

In [2]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

In [19]:
data = pd.read_csv('../Data/preprocessed.csv')
data.head()

,Unnamed: 0,article_id,title,text,unique_tokens_rate,num_hrefs,num_imgs,num_videos,average_token_length,num_keywords,...,april opens,fire tv,new window,new york,opens new,pic twitter,see also,seealso slug,social media.1,twitter com
0,0,7014291,It's Time for a YouTube Competitor,It was reported last week that Yahoo is gettin...,0.640816,5.0,3.0,1.0,4.579592,5,...,0,0,0,0,0,0,0,1,0,0
1,1,7014327,The Ultimate Roundup of 2014's Biggest April F...,April Fools' marks the day when pranking goes ...,0.549618,4.0,1.0,0.0,4.636704,7,...,0,0,0,0,0,0,0,0,0,0
2,2,7014304,A Selfie and One Garish Jacket: How the Red So...,The 2013 World Series champion Boston Red Sox ...,0.577236,8.0,1.0,0.0,4.627451,7,...,5,0,17,0,13,5,0,0,0,5
3,3,7031995,African Telecom Proposes More Diverse Emoticons,Less than a week after Apple pledged to make i...,0.525896,4.0,1.0,0.0,5.083665,9,...,0,0,2,0,2,0,1,1,0,0
4,4,6541359,"Google to Launch Improved Android Camera App, ...",Android users may soon see a series of new fea...,0.588889,6.0,9.0,0.0,4.336022,7,...,0,0,1,0,1,0,0,1,0,0


In [20]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [21]:
data.head()

,article_id,title,text,unique_tokens_rate,num_hrefs,num_imgs,num_videos,average_token_length,num_keywords,min_avg_key,...,april opens,fire tv,new window,new york,opens new,pic twitter,see also,seealso slug,social media.1,twitter com
0,7014291,It's Time for a YouTube Competitor,It was reported last week that Yahoo is gettin...,0.640816,5.0,3.0,1.0,4.579592,5,1459.500,...,0,0,0,0,0,0,0,1,0,0
1,7014327,The Ultimate Roundup of 2014's Biggest April F...,April Fools' marks the day when pranking goes ...,0.549618,4.0,1.0,0.0,4.636704,7,0.000,...,0,0,0,0,0,0,0,0,0,0
2,7014304,A Selfie and One Garish Jacket: How the Red So...,The 2013 World Series champion Boston Red Sox ...,0.577236,8.0,1.0,0.0,4.627451,7,2270.889,...,5,0,17,0,13,5,0,0,0,5
3,7031995,African Telecom Proposes More Diverse Emoticons,Less than a week after Apple pledged to make i...,0.525896,4.0,1.0,0.0,5.083665,9,0.000,...,0,0,2,0,2,0,1,1,0,0
4,6541359,"Google to Launch Improved Android Camera App, ...",Android users may soon see a series of new fea...,0.588889,6.0,9.0,0.0,4.336022,7,3264.605,...,0,0,1,0,1,0,0,1,0,0


In [22]:
# extracting numeric data into dataframe df
df = data.drop(['article_id', 'title', 'text', 'data_channel_cleaned', 'text_prepared', 'published_date_cleaned'], axis=1)
df.head()

,unique_tokens_rate,num_hrefs,num_imgs,num_videos,average_token_length,num_keywords,min_avg_key,max_avg_key,avg_avg_key,href_avg_shares,...,april opens,fire tv,new window,new york,opens new,pic twitter,see also,seealso slug,social media.1,twitter com
0,0.640816,5.0,3.0,1.0,4.579592,5,1459.500,4817.692,3370.415,1833.333,...,0,0,0,0,0,0,0,1,0,0
1,0.549618,4.0,1.0,0.0,4.636704,7,0.000,3581.094,2556.367,8500.000,...,0,0,0,0,0,0,0,0,0,0
2,0.577236,8.0,1.0,0.0,4.627451,7,2270.889,3852.800,3018.167,2076.333,...,5,0,17,0,13,5,0,0,0,5
3,0.525896,4.0,1.0,0.0,5.083665,9,0.000,3581.094,1871.403,2200.000,...,0,0,2,0,2,0,1,1,0,0
4,0.588889,6.0,9.0,0.0,4.336022,7,3264.605,8453.451,5155.323,805.000,...,0,0,1,0,1,0,0,1,0,0


In [23]:
df.shape

(1003, 36)

In [24]:
# splitting dataset into predictor and target variables
X = df.drop('shares',axis=1)
y = df[['shares']]

In [25]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
print((x_train.shape, x_test.shape), (y_train.shape,y_test.shape))

((802, 35), (201, 35)) ((802, 1), (201, 1))


In [26]:
scaler = StandardScaler()

In [27]:
x_train = scaler.fit_transform(x_train)

In [31]:
x_train

array([[ 1.07789591, -0.00784462, -0.59958131, ..., -0.77720135,
        -0.21205872, -0.21541484],
       [-0.31869803,  2.88619426, -0.59958131, ...,  1.27317852,
        -0.21205872, -0.21541484],
       [ 1.50333135, -0.63698351, -0.42642171, ..., -0.77720135,
        -0.21205872, -0.21541484],
       ...,
       [-0.45344529,  0.36963871, -0.59958131, ..., -0.77720135,
         5.18702352, -0.21541484],
       [ 0.04649706, -0.76281129, -0.77274092, ...,  1.27317852,
        -0.21205872, -0.21541484],
       [ 1.23317868, -0.76281129, -0.59958131, ..., -0.77720135,
        -0.21205872, -0.21541484]])

In [32]:
x_test = scaler.transform(x_test)

<h1>Models</h1>

In [33]:
# Fitting the Linear Regression model
lm = LinearRegression()
lm.fit(x_train, y_train)

LinearRegression()

In [34]:
lm.score(x_test, y_test)

0.05873832261827805

In [35]:
y_hat = lm.predict(x_test)

print("Test R2 score: "+ str(r2_score(y_test,y_hat)))
print("Test MAE: "+ str(mean_absolute_error(y_test,y_hat)))
print("Test RMSE: "+ str(np.sqrt(mean_squared_error(y_test,y_hat))))

Test R2 score: 0.05873832261827805
Test MAE: 2130.807247358031
Test RMSE: 3026.825541651812


In [36]:
rm = Ridge()
rm.fit(x_train, y_train)

Ridge()

In [37]:
rm.score(x_test, y_test)

0.05994034285274308

In [38]:
y_hat = rm.predict(x_test)

print("Test R2 score: "+ str(r2_score(y_test,y_hat)))
print("Test MAE: "+ str(mean_absolute_error(y_test,y_hat)))
print("Test RMSE: "+ str(np.sqrt(mean_squared_error(y_test,y_hat))))

Test R2 score: 0.05994034285274308
Test MAE: 2129.4766390039927
Test RMSE: 3024.892249383508


In [39]:
mlp = MLPRegressor(random_state=42)

In [40]:
mlp.fit(x_train, y_train)

C:\Users\ituser\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ituser\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(random_state=42)

In [41]:
y_hat = mlp.predict(x_test)

In [42]:
mlp.score(x_test, y_test)

-0.7317615431060098

In [43]:
y_hat = mlp.predict(x_test)

print("Test R2 score: "+ str(r2_score(y_test,y_hat)))
print("Test MAE: "+ str(mean_absolute_error(y_test,y_hat)))
print("Test RMSE: "+ str(np.sqrt(mean_squared_error(y_test,y_hat))))

Test R2 score: -0.7317615431060098
Test MAE: 2688.3843138811794
Test RMSE: 4105.596335536841
